In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pywt  # Para usar DWT
from scipy.signal import butter, filtfilt, iirnotch, medfilt

# Discrete Wavelet Transform (DWT) para denoising y reconstrucción
def dwt_denoising(signal, wavelet='db4', level=4):
    coeffs = pywt.wavedec(signal, wavelet, level=level)
    # Aplicar threshold a los coeficientes detallados (umbral suave)
    threshold = np.sqrt(2 * np.log(len(signal))) * np.median(np.abs(coeffs[-1])) / 0.6745
    coeffs = [pywt.threshold(c, threshold, mode='soft') for c in coeffs]
    # Reconstruir la señal denoised
    return pywt.waverec(coeffs, wavelet)

# Función para calcular SNR
def calculate_snr(original, denoised):
    signal_power = np.sum(original**2)
    noise_power = np.sum((original - denoised)**2)
    return 10 * np.log10(signal_power / noise_power)

# Función para calcular PRD
def calculate_prd(original, denoised):
    return 100 * np.sqrt(np.sum((original - denoised) ** 2) / np.sum(original ** 2))

# Función para calcular RMSE
def calculate_rmse(original, denoised):
    return np.sqrt(np.mean((original - denoised) ** 2))

# Función para plotear FFT y guardar el gráfico
def plot_fft(signal, fs, title="FFT", save_name=None):
    N = len(signal)
    fft_values = np.fft.fft(signal)
    fft_magnitude = np.abs(fft_values)[:N//2]
    fft_frequencies = np.fft.fftfreq(N, 1/fs)[:N//2]
    fft_magnitude_db = 20 * np.log10(fft_magnitude / np.max(fft_magnitude))

    plt.figure(figsize=(10, 4))
    plt.plot(fft_frequencies, fft_magnitude_db)
    plt.grid(True)
    plt.xlabel("Frecuencia (Hz)")
    plt.ylabel("Magnitud (dB)")
    plt.title(title)
    plt.xlim([0, 200])  # Ajusta el límite de frecuencia según sea necesario
    plt.xticks(np.arange(0, 200, 10))
    
    # Guardar gráfico si se proporciona un nombre de archivo
    if save_name:
        plt.savefig(save_name, format='png')
    
    plt.close()  # Cerrar la figura después de guardar

# Función para graficar y comparar señales originales y denoised
def plot_comparison(original_signal, denoised_signal, fs, start_time, end_time, title="Comparación de Señales", save_name=None):
    start_sample = int(start_time * fs)  # Convertir tiempo a muestras
    end_sample = int(end_time * fs)
    
    # Cortar las señales para el rango especificado
    original_segment = original_signal[start_sample:end_sample]
    denoised_segment = denoised_signal[start_sample:end_sample]

    # Graficar
    plt.figure(figsize=(12, 6))
    plt.plot(original_segment, label="Señal Original", color='blue')
    plt.plot(denoised_segment, label="Señal Denoised", color='red', linestyle='--')
    plt.title(title + f" (Segmento {start_time}-{end_time}s)")
    plt.xlabel("Muestras")
    plt.ylabel("Amplitud")
    plt.legend()
    plt.grid(True)
    
    # Guardar gráfico si se proporciona un nombre de archivo
    if save_name:
        plt.savefig(save_name, format='png')

    plt.close()  # Cerrar la figura después de guardar

# Parámetros de muestreo
Fs = 1000  # Frecuencia de muestreo
Ts = 1 / Fs  # Período de muestreo

# Definir el intervalo de tiempo (en segundos)
start_time = 4.2  # Inicio del tiempo deseado
end_time = 5    # Final del tiempo deseado

# Lista de archivos a procesar
file_paths = [
    "C:\\Users\\Ingrid\\Downloads\\Señales_2\\New_signal\\Estado_basal_Toma_2_I_deriv.txt",
    "C:\\Users\\Ingrid\\Downloads\\Señales_2\\New_signal\\Estado_sin_Respiracion_I_deriv.txt",
    "C:\\Users\\Ingrid\\Downloads\\Señales_2\\New_signal\\Estado_con_Respiracion_I_deriv.txt",
    "C:\\Users\\Ingrid\\Downloads\\Señales_2\\New_signal\\Ejercicio_I_deriv.txt",
    "C:\\Users\\Ingrid\\Downloads\\Señales_2\\New_signal\\Simulacion_60_bpm.txt",
    "C:\\Users\\Ingrid\\Downloads\\Señales_2\\New_signal\\Simulacion_150bpm.txt"
]

nombres = ["Estado Basal (1ra derivación)", "Estado sin respiración (1ra derivación)", "Estado con respiración (1ra derivación)",
           "Ejercicio (1ra derivación)", "Simulación a 60 bpm", "Simulación a 150 bpm"]

# Procesar cada archivo
for idx, file_path in enumerate(file_paths):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    data_start = None
    for i, line in enumerate(lines):
        if 'EndOfHeader' in line:
            data_start = i + 1
            break

    if data_start is None:
        raise ValueError(f"No se encontró 'EndOfHeader' en el archivo {file_path}.")

    # Extraer las líneas de datos y convertirlas en un array de NumPy
    data_lines = lines[data_start:]
    data = np.array([list(map(float, line.strip().split('\t'))) for line in data_lines])

    # Extraer la señal EMG (última columna en los datos)
    signal1 = data[:, 5]  # Usar la columna correcta según los datos
    
    # Aplicar DWT para denoising
    denoised_signal = dwt_denoising(signal1)

    # Calcular métricas de performance global
    snr_value_global = calculate_snr(signal1, denoised_signal)
    prd_value_global = calculate_prd(signal1, denoised_signal)
    rmse_value_global = calculate_rmse(signal1, denoised_signal)

    print(f"{nombres[idx]} - Global - SNR: {snr_value_global:.2f} dB, PRD: {prd_value_global:.2f}%, RMSE: {rmse_value_global:.4f}")

    # Calcular métricas de performance para el segmento de 2 a 5 segundos
    start_sample = int(start_time * Fs)
    end_sample = int(end_time * Fs)

    signal1_segment = signal1[start_sample:end_sample]
    denoised_segment = denoised_signal[start_sample:end_sample]

    # Métricas para el segmento 2-5s
    snr_value_segment = calculate_snr(signal1_segment, denoised_segment)
    prd_value_segment = calculate_prd(signal1_segment, denoised_segment)
    rmse_value_segment = calculate_rmse(signal1_segment, denoised_segment)

    print(f"{nombres[idx]} (4.2-5s) - SNR: {snr_value_segment:.2f} dB, PRD: {prd_value_segment:.2f}%, RMSE: {rmse_value_segment:.4f}")

    # Comparar las señales original y denoised para la señal completa
    plot_comparison(signal1, denoised_signal, Fs, 0, len(signal1) / Fs, title=f"Comparación {nombres[idx]}", save_name=f"comparacion_{nombres[idx]}.png")

    # Comparar las señales original y denoised solo en el segmento de 4 a 5 segundos
    plot_comparison(signal1, denoised_signal, Fs, 4.2, 5, title=f"Comparación {nombres[idx]} (4.2-5s)", save_name=f"comparacion_{nombres[idx]}_4.2-5s.png")

    # Graficar la FFT global
    plot_fft(denoised_signal, Fs, title=f"FFT {nombres[idx]}", save_name=f"fft_{nombres[idx]}.png")

    # Graficar la FFT del segmento de 2 a 5 segundos
    plot_fft(denoised_segment, Fs, title=f"FFT {nombres[idx]} (4.2-5s)", save_name=f"fft_{nombres[idx]}_4.2-5s.png")



Estado Basal (1ra derivación) - Global - SNR: 45.96 dB, PRD: 0.50%, RMSE: 2.5563
Estado Basal (1ra derivación) (4.2-5s) - SNR: 46.07 dB, PRD: 0.50%, RMSE: 2.5619
Estado sin respiración (1ra derivación) - Global - SNR: 46.00 dB, PRD: 0.50%, RMSE: 2.5726
Estado sin respiración (1ra derivación) (4.2-5s) - SNR: 45.94 dB, PRD: 0.50%, RMSE: 2.5062
Estado con respiración (1ra derivación) - Global - SNR: 48.35 dB, PRD: 0.38%, RMSE: 1.9414
Estado con respiración (1ra derivación) (4.2-5s) - SNR: 48.07 dB, PRD: 0.39%, RMSE: 2.0036
Ejercicio (1ra derivación) - Global - SNR: 46.97 dB, PRD: 0.45%, RMSE: 2.2795
Ejercicio (1ra derivación) (4.2-5s) - SNR: 46.17 dB, PRD: 0.49%, RMSE: 2.4997
Simulación a 60 bpm - Global - SNR: 51.91 dB, PRD: 0.25%, RMSE: 1.2886
Simulación a 60 bpm (4.2-5s) - SNR: 52.00 dB, PRD: 0.25%, RMSE: 1.2729
Simulación a 150 bpm - Global - SNR: 52.85 dB, PRD: 0.23%, RMSE: 1.1599
Simulación a 150 bpm (4.2-5s) - SNR: 52.66 dB, PRD: 0.23%, RMSE: 1.1846
